<a href="https://colab.research.google.com/github/EUDA289/mulcam_b_4/blob/master/call_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
call_rate={}
date_list=[]
rate_list=[]

## 90~542
total_page=544
for page in tqdm(range(total_page-90)):  
  url='https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(page+90)
  resp=requests.get(url)
  soup=BeautifulSoup(resp.content, 'lxml')
  date=soup.select('tbody tr td.date')
  for i in range(7):
    date_list.append(date[i].get_text().replace('\n', " ").replace('\t', " ").strip())

for page in tqdm(range(total_page-90)):  
  url='https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(page+90)
  resp=requests.get(url)
  soup=BeautifulSoup(resp.content, 'lxml')
  rate=soup.select('tbody tr td.num')
  for i in range(0, 21, 3):
    rate_list.append(rate[i].get_text().replace('\n', " ").replace('\t', " ").strip())

## 데이터프레임 만들기

In [ ]:
call_rate['date']=date_list
call_rate['rate']=rate_list

call_rate_df=pd.DataFrame(call_rate)

In [ ]:
call_rate_df

,date,rate
0,2018.01.11,1.50
1,2018.01.10,1.51
2,2018.01.09,1.49
3,2018.01.08,1.46
4,2018.01.05,1.47
...,...,...
3173,2005.04.07,3.23
3174,2005.04.06,3.23
3175,2005.04.04,3.24
3176,2005.04.01,3.25


In [ ]:
call_rate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3178 entries, 0 to 3177
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3178 non-null   object
 1   rate    3178 non-null   object
dtypes: object(2)
memory usage: 49.8+ KB


In [ ]:
call_rate_df['date']=pd.to_datetime(call_rate_df['date'])

In [ ]:
call_rate_df['rate']=call_rate_df['rate'].astype(str).astype(float)

In [ ]:
call_rate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3178 entries, 0 to 3177
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3178 non-null   datetime64[ns]
 1   rate    3178 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 49.8 KB


In [ ]:
call_rate_df.head(20)

,date,rate
0,2018-01-11,1.50
1,2018-01-10,1.51
2,2018-01-09,1.49
3,2018-01-08,1.46
4,2018-01-05,1.47
5,2018-01-04,1.48
6,2018-01-03,1.50
7,2018-01-02,1.52
8,2017-12-29,1.58
9,2017-12-28,1.55


In [ ]:
# call_rate_bok=call_rate_df[8:3157]

In [ ]:
# call_rate_bok.head(10)

In [ ]:
# call_rate_bok.to_csv('./drive/My Drive/Colab Notebooks/NLP/프로젝트/call_rate.csv', index=False
#                                             )

## nan 처리

In [ ]:
import datetime
from datetime import timedelta

In [ ]:
# null_dic={}
# null_date=[]
# null_rate=[]

# for i in range(8, 3157, 1):
#   if call_rate_df['date'][i] - call_rate_df['date'][i+1] == timedelta(days=1):
#     pass
#   else: 
#     null_date.append(call_rate_bok['date'][i] - timedelta(days=1))
#     null_rate.append(np.nan)



In [ ]:
# if call_rate_df['date'][1] -  call_rate_df['date'][2] == timedelta(days=1):
#   print('yes!')

In [ ]:
# null_dic['date']=null_date
# null_dic['rate']=null_rate

# null_df=pd.DataFrame(null_dic)

In [ ]:
# null_df

In [ ]:
call_rate_df['date'][8] in list(call_rate_df['date'])
call_rate_df['date'][11] - timedelta(days=1) in list(call_rate_df['date'])

False

In [ ]:
null_dic={}
null_date=[]
null_rate=[]

day = call_rate_df['date'][0]

while 1:
  if day == call_rate_df['date'][3177]:
    break

  day = day - timedelta(days=1) 

  if day in list(call_rate_df['date']):
    continue

  else:
    null_date.append(day)
    null_rate.append(np.nan)


In [ ]:
null_dic['date']=null_date
null_dic['rate']=null_rate

null_df=pd.DataFrame(null_dic)

In [ ]:
null_df

,date,rate
0,2018-01-07,NaN
1,2018-01-06,NaN
2,2018-01-01,NaN
3,2017-12-31,NaN
4,2017-12-30,NaN
...,...,...
1487,2005-04-10,NaN
1488,2005-04-09,NaN
1489,2005-04-05,NaN
1490,2005-04-03,NaN


In [ ]:
call_rate_data=pd.concat([call_rate_df,null_df], ignore_index=True)

In [ ]:
call_rate_data=call_rate_data.sort_values(by='date', ascending=False, ignore_index=True)
call_rate_data

,date,rate
0,2018-01-11,1.50
1,2018-01-10,1.51
2,2018-01-09,1.49
3,2018-01-08,1.46
4,2018-01-07,NaN
...,...,...
4665,2005-04-04,3.24
4666,2005-04-03,NaN
4667,2005-04-02,NaN
4668,2005-04-01,3.25


## 열 추가

### 한달 전 날짜 및 콜금리

In [ ]:
call_rate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4670 entries, 0 to 4669
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4670 non-null   datetime64[ns]
 1   rate    3178 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 73.1 KB


In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
edt =call_rate_data['date'][8] -  relativedelta(months=1)
edt=np.datetime64(edt)
edt

numpy.datetime64('2017-12-03T00:00:00.000000')

In [ ]:
call_rate_data[call_rate_data['date']==edt]['rate'].values

array([1.26])

In [ ]:
len(call_rate_data[call_rate_data['date']==edt]['rate'].values)

1

In [ ]:
call_rate_data['date'].index

RangeIndex(start=0, stop=4670, step=1)

In [ ]:
call_rate_bok=call_rate_data[11:4639]
call_rate_bok

,date,rate
11,2017-12-31,NaN
12,2017-12-30,NaN
13,2017-12-29,1.58
14,2017-12-28,1.55
15,2017-12-27,1.51
...,...,...
4634,2005-05-05,NaN
4635,2005-05-04,3.21
4636,2005-05-03,3.30
4637,2005-05-02,3.29


In [ ]:
# ex_call_rate={}
# ex_date=[]
# ex_rate=[]

# for i in range(11, 3178, 1):
#   edt = np.datetime64(call_rate_df['date'][i] - relativedelta(months=1))
#   ex_date.append(edt)
#   if len(call_rate_df[call_rate_df['date']==edt]['rate'].values) ==0:
#     exr = np.nan
#     ex_rate.append(exr)
#   else:
#     exr=call_rate_df[call_rate_df['date']==edt]['rate'].values[0]
#     ex_rate.append(exr)

# ex_call_rate['date_MoM']=ex_date
# ex_call_rate['rate_MoM']=ex_rate
# ex_call_rate_df=pd.DataFrame(ex_call_rate)

In [ ]:
# ex_call_rate_df

In [ ]:
edt = np.datetime64(call_rate_data['date'][17]- relativedelta(months=1))
edt

numpy.datetime64('2017-11-25T00:00:00.000000')

In [ ]:
call_rate_data[call_rate_data['date']==edt]['rate'].values[0]

nan

In [ ]:
ex_call_rate={}
ex_date=[]
ex_rate=[]

for i in range(11, 4639, 1):
  edt = np.datetime64(call_rate_data['date'][i] - relativedelta(months=1))
  ex_date.append(edt)
  # exr = call_rate_data[call_rate_data['date']==edt]['rate'].values[0]

  if call_rate_data[call_rate_data['date']==edt]['rate'].values[0] >= 0:
    exr=call_rate_data[call_rate_data['date']==edt]['rate'].values[0]
  # elif call_rate_data[call_rate_data['date']==edt]['rate'].values[0] !=0:
  #   edt=np.datetime64(call_rate_data['date'][i+1] - relativedelta(months=1))
  #   exr=call_rate_data[call_rate_data['date']==edt]['rate'].values[0]
  else:
    edt=np.datetime64(call_rate_data['date'][i+1] - relativedelta(months=1))
    if call_rate_data[call_rate_data['date']==edt]['rate'].values[0] >= 0:
      exr=call_rate_data[call_rate_data['date']==edt]['rate'].values[0]
    else:
      edt=np.datetime64(call_rate_data['date'][i+2] - relativedelta(months=1))
      if call_rate_data[call_rate_data['date']==edt]['rate'].values[0] >= 0:
        exr=call_rate_data[call_rate_data['date']==edt]['rate'].values[0]
      else:
        edt=np.datetime64(call_rate_data['date'][i+3] - relativedelta(months=1))
        if call_rate_data[call_rate_data['date']==edt]['rate'].values[0] >= 0:
          exr=call_rate_data[call_rate_data['date']==edt]['rate'].values[0]
        else: 
          edt=np.datetime64(call_rate_data['date'][i+4] - relativedelta(months=1))
          if call_rate_data[call_rate_data['date']==edt]['rate'].values[0] >= 0:
            exr=call_rate_data[call_rate_data['date']==edt]['rate'].values[0]
          else:
            edt=np.datetime64(call_rate_data['date'][i+5] - relativedelta(months=1))
            if call_rate_data[call_rate_data['date']==edt]['rate'].values[0] >= 0: 
              exr=call_rate_data[call_rate_data['date']==edt]['rate'].values[0]
            else: 
              edt=np.datetime64(call_rate_data['date'][i+6] - relativedelta(months=1))
              if call_rate_data[call_rate_data['date']==edt]['rate'].values[0] >= 0: 
                exr=call_rate_data[call_rate_data['date']==edt]['rate'].values[0]
              else:
                exr=1.27
      
      

  ex_rate.append(exr)

ex_call_rate['date_MoM']=ex_date
ex_call_rate['rate_MoM']=ex_rate
ex_call_rate_df=pd.DataFrame(ex_call_rate)

In [ ]:
ex_call_rate_df[ex_call_rate_df['rate_MoM'].isnull()==True]

,date_MoM,rate_MoM


In [ ]:
call_rate_bok.shape, ex_call_rate_df.shape

((4628, 2), (4628, 2))

In [ ]:
call_rate_bok

,date,rate
11,2017-12-31,NaN
12,2017-12-30,NaN
13,2017-12-29,1.58
14,2017-12-28,1.55
15,2017-12-27,1.51
...,...,...
4634,2005-05-05,NaN
4635,2005-05-04,3.21
4636,2005-05-03,3.30
4637,2005-05-02,3.29


In [ ]:
call_rate_bok=call_rate_bok.reset_index()

In [ ]:
semi_final_data = pd.concat([call_rate_bok, ex_call_rate_df], axis=1)

In [ ]:
semi_final_data=semi_final_data.drop(['index'], axis=1)

In [ ]:
semi_final_data

,date,rate,date_MoM,rate_MoM
0,2017-12-31,NaN,2017-11-30,1.53
1,2017-12-30,NaN,2017-11-30,1.53
2,2017-12-29,1.58,2017-11-29,1.26
3,2017-12-28,1.55,2017-11-28,1.26
4,2017-12-27,1.51,2017-11-27,1.26
...,...,...,...,...
4623,2005-05-05,NaN,2005-04-05,3.24
4624,2005-05-04,3.21,2005-04-04,3.24
4625,2005-05-03,3.30,2005-04-03,3.25
4626,2005-05-02,3.29,2005-04-02,3.25


In [ ]:
# semi_final_data.to_csv('./drive/My Drive/Colab Notebooks/NLP/프로젝트/semi_final_data.csv', index=False)

In [ ]:
final_data=semi_final_data.dropna()

In [ ]:
final_data

,date,rate,date_MoM,rate_MoM
2,2017-12-29,1.58,2017-11-29,1.26
3,2017-12-28,1.55,2017-11-28,1.26
4,2017-12-27,1.51,2017-11-27,1.26
5,2017-12-26,1.52,2017-11-26,1.25
9,2017-12-22,1.51,2017-11-22,1.24
...,...,...,...,...
4619,2005-05-09,3.29,2005-04-09,3.26
4622,2005-05-06,3.27,2005-04-06,3.23
4624,2005-05-04,3.21,2005-04-04,3.24
4625,2005-05-03,3.30,2005-04-03,3.25


In [ ]:
final_data=final_data.reset_index()

In [ ]:
final_data=final_data.drop(['index'], axis=1)

In [ ]:
final_data

,date,rate,date_MoM,rate_MoM
0,2017-12-29,1.58,2017-11-29,1.26
1,2017-12-28,1.55,2017-11-28,1.26
2,2017-12-27,1.51,2017-11-27,1.26
3,2017-12-26,1.52,2017-11-26,1.25
4,2017-12-22,1.51,2017-11-22,1.24
...,...,...,...,...
3144,2005-05-09,3.29,2005-04-09,3.26
3145,2005-05-06,3.27,2005-04-06,3.23
3146,2005-05-04,3.21,2005-04-04,3.24
3147,2005-05-03,3.30,2005-04-03,3.25


In [ ]:
up_down_dic={}
up_down_list=[]


for i, j in zip(list(final_data['rate']), list(final_data['rate_MoM'])):
  if i >= j:
    if i==j:
      up_down_list.append('even')
    else:
      up_down_list.append('up')
  else:
    up_down_list.append('down')

up_down_dic['up_down']=up_down_list

In [ ]:
len(up_down_list)

3149

In [ ]:
up_down_col=pd.DataFrame(up_down_dic)

In [ ]:
up_down_col

,up_down
0,up
1,up
2,up
3,up
4,up
...,...
3144,up
3145,up
3146,down
3147,up


In [ ]:
final_data=pd.concat([final_data, up_down_col], axis=1)
final_data

,date,rate,date_MoM,rate_MoM,up_down
0,2017-12-29,1.58,2017-11-29,1.26,up
1,2017-12-28,1.55,2017-11-28,1.26,up
2,2017-12-27,1.51,2017-11-27,1.26,up
3,2017-12-26,1.52,2017-11-26,1.25,up
4,2017-12-22,1.51,2017-11-22,1.24,up
...,...,...,...,...,...
3144,2005-05-09,3.29,2005-04-09,3.26,up
3145,2005-05-06,3.27,2005-04-06,3.23,up
3146,2005-05-04,3.21,2005-04-04,3.24,down
3147,2005-05-03,3.30,2005-04-03,3.25,up


In [ ]:
# final_data.to_csv('./drive/My Drive/Colab Notebooks/NLP/프로젝트/final_data.csv', index=False)